# Modelo GPT - Parte 1

## [Tópicos em Ciência de Dados](https://denmartins.github.io/teaching/2025-topicos-cd)

### [Prof. Dr. Denis Mayr Lima Martins](https://denmartins.github.io/)

### [Pontifícia Universidade Católica de Campinas](https://www.puc-campinas.edu.br/)

<img src="https://www.puc-campinas.edu.br/wp-content/uploads/2022/06/logo-puc.png" width="100px"/>


# Objetivos de Aprendizagem

- Implementar um modelo de LLM semelhante ao GPT que pode ser treinado para gerar texto.
- Compreender o conceito de normalização de camadas e sua importância no treinamento de redes neurais.
<!-- - Entender como conexões de atalho (skipping connections) em redes neurais profundas ajudam no treinamento.
- Implementar blocos Transformer para criar modelos GPT de diferentes tamanhos -->

<center>
<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
Baseado no Livro <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> de <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="200px"></a>
</td>
</tr>
</table>
</center>

## Implementando um Modelo GPT (Parte 1)

Com o mecanismo de atenção disponível, agora iremos programar os outros blocos de construção de um LLM e montá-los em um modelo do tipo GPT que pode ser treinado para gerar texto convincente.


In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

- `import os`: Importa o módulo *os* para interagir com o sistema operacional.  
- `os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"`: Define uma variável de ambiente que instrui a biblioteca MKL da Intel a permitir que múltiplas instâncias da mesma biblioteca sejam carregadas simultaneamente, sem gerar erro. Tal configuração costuma ser empregada para resolver conflitos entre bibliotecas em tarefas de computação científica ou aprendizado de máquina.

In [2]:
from importlib.metadata import version

import matplotlib
import tiktoken
import torch

print("matplotlib version:", version("matplotlib"))
print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

matplotlib version: 3.10.5
torch version: 2.8.0
tiktoken version: 0.11.0


## 1. Codificando uma Arquitetura de LLM

- Modelos de linguagem de larga escala (LLMs), como o GPT, são arquiteturas de redes neurais profundas projetadas para gerar novo texto palavra (ou token) por palavra. Apesar do tamanho, a arquitetura do modelo é menos complicada do que se imagina, já que muitos de seus componentes são repetidos.
- A arquitetura de um GPT contém, ao lado das camadas de embedding, blocos *transformer* que incluem o módulo de atenção multi-cabeças mascarada implementado anteriormente.


In [3]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length, which might be much larger in other models
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers/transformer blocks
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}

- **`"vocab_size"`** tamanho de vocabulário de 50257 palavras, suportado pelo tokenizador BPE.  
- **`"context_length"`** contagem máxima de tokens de entrada do modelo, conforme habilitado pelos embeddings posicionais.  
- **`"emb_dim"`** tamanho do embedding para os tokens de entrada, convertendo cada token de entrada num vetor de 768 dimensões.  
- **`"n_heads"`** número de cabeças de atenção no mecanismo de multi‑head attention.  
- **`"n_layers"`** número de blocos transformer dentro do modelo, que iremos implementar em breve.  
- **`"drop_rate"`** descarta 10% das unidades ocultas durante o treinamento para mitigar overfitting.  
- **`"qkv_bias"`** decide se as camadas `Linear` no mecanismo de MHA devem incluir um vetor de bias ao calcular os tensores query (Q), key (K) e value (V); desativaremos essa opção, prática padrão em LLMs modernos.

## Implementação Principal

Arquitetura inicial chamada que serve como esqueleto do modelo.

In [ ]:
import torch
import torch.nn as nn

class SimpleGPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        # a camada de entrada mapeia do tamanho do vocabulário para o espaço de embedding
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])
        # Use um placeholder para TransformerBlock
        self.trf_blocks = nn.Sequential(*[SimpleTransformerBlock(cfg) for _ in range(cfg["n_layers"])])
        # Use um placeholder para LayerNorm
        self.final_norm = SimpleLayerNorm(cfg["emb_dim"])
        # camada de saída, mapeia do espaço de embedding de volta ao tamanho do vocabulário
        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)
    
    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

## Camadas adicionais

In [5]:
class SimpleTransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        # Um placeholder simples

    def forward(self, x):
        # Este bloco não faz nada e apenas retorna sua entrada.
        return x

class SimpleLayerNorm(nn.Module):
    def __init__(self, normalized_shape, eps=1e-5):
        super().__init__()
        # Os parâmetros aqui são apenas para imitar a interface do LayerNorm.

    def forward(self, x):
        # Esta camada não faz nada e apenas retorna sua entrada.
        return x

A classe `SimpleGPTModel` presente neste código define uma versão simplificada de um modelo do tipo GPT utilizando o módulo de redes neurais do PyTorch (`nn.Module`). A arquitetura do modelo na classe `SimpleGPTModel` consiste em embeddings de token e posicional, dropout, uma série de blocos transformadores (`SimpleTransformerBlock`), normalização de camada final (`SimpleLayerNorm`) e uma camada linear de saída (out_head). A configuração é passada por meio de um dicionário Python; por exemplo, o dicionário `GPT_CONFIG_124M` que criamos anteriormente.  

O método `forward` descreve o fluxo de dados através do modelo: ele calcula os embeddings de token e posicional para os índices de entrada, aplica dropout, processa os dados pelos blocos transformadores, aplica a normalização e, finalmente, gera logits com a camada linear de saída.  

O código já é funcional, como veremos mais adiante após prepararmos os dados de entrada. Entretanto, atualmente utilizamos placeholders (`SimpleLayerNorm` e `SimpleTransformerBlock`) para o bloco transformador e a normalização de camada.  

Em seguida, prepararemos os dados de entrada e inicializaremos um novo modelo GPT para ilustrar seu uso. Construindo sobre as figuras que vimos no episódio em que codificamos o tokenizador, a figura abaixo fornece uma visão geral de alto nível de como os dados entram e saem de um modelo GPT.

## Visão Geral

<img src="https://camo.githubusercontent.com/e065aaca707d5039aea24d8752be69ab78a7b19028724cfacf0902d8b46ae9ed/68747470733a2f2f73656261737469616e72617363686b612e636f6d2f696d616765732f4c4c4d732d66726f6d2d736372617463682d696d616765732f636830345f636f6d707265737365642f30342e776562703f313233" width="500"/>

## Tokenização

A saída do código abaixo é o que a LLM recebe, e a tarefa consiste em produzir a próxima palavra desse texto.

In [6]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

batch = []

txt1 = "Every effort moves you"  # every word will result in a token
txt2 = "Every day holds a"

batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim=0)
print(batch)

tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])


## Instanciando o Modelo

Inicializamos uma nova instância do `SimpleGPTModel` com 124 milhões de parâmetros conforme especificado acima e alimentamos o modelo com o lote tokenizado (os resultados do modelo são comumente denominados *logits*).

In [7]:
torch.manual_seed(123)
model = SimpleGPTModel(GPT_CONFIG_124M)  
logits = model(batch)                   
print("Output shape:", logits.shape)
print(logits)

Output shape: torch.Size([2, 4, 50257])
tensor([[[-1.2034,  0.3201, -0.7130,  ..., -1.5548, -0.2390, -0.4667],
         [-0.1192,  0.4539, -0.4432,  ...,  0.2392,  1.3469,  1.2430],
         [ 0.5307,  1.6720, -0.4695,  ...,  1.1966,  0.0111,  0.5835],
         [ 0.0139,  1.6754, -0.3388,  ...,  1.1586, -0.0435, -1.0400]],

        [[-1.0908,  0.1798, -0.9484,  ..., -1.6047,  0.2439, -0.4530],
         [-0.7860,  0.5581, -0.0610,  ...,  0.4835, -0.0077,  1.6621],
         [ 0.3567,  1.2698, -0.6398,  ..., -0.0162, -0.1296,  0.3717],
         [-0.2407, -0.7349, -0.5102,  ...,  2.0057, -0.3694,  0.1814]]],
       grad_fn=<UnsafeViewBackward0>)


O tensor de saída possui duas linhas correspondentes às duas amostras de texto. Cada amostra consiste em 4 tokens (um para cada palavra); cada token é um vetor de 50257 dimensões, o que corresponde ao tamanho do vocabulário do tokenizador.

Embeddings tem 50257 dimensões porque cada uma dessas dimensões representa um token único no vocabulário. Ao final deste episódio, quando implementarmos o código de pós‑processamento, converteremos esses vetores de 50257 dimensões de volta em IDs de token, que então poderemos decodificar em palavras.

## 2. Layer Normalization


Usamos a normalização de camada para melhorar a estabilidade e eficiência do treinamento de redes neurais. 
- **Ideia central**: ajustar as ativações (saídas) de uma camada de rede neural de modo que tenham média zero e variância unitária, também conhecida como *unit variance*. 
- **Vantagem**: Acelera a convergência para pesos efetivos e garante um treinamento consistente e confiável. 
- Nas arquiteturas GPT‑2 e nos transformers, a normalização de camada costuma ser aplicada antes e depois do módulo de atenção multi‑cabeça e antes da camada de saída final.


![](https://camo.githubusercontent.com/1bb0018e68d16529b969ccc6b0bca371ed5a7b9a514d1bd3f345c4941c0f463d/68747470733a2f2f73656261737469616e72617363686b612e636f6d2f696d616765732f4c4c4d732d66726f6d2d736372617463682d696d616765732f636830345f636f6d707265737365642f30352e77656270)

Vamos observar como funciona a normalização de camada passando uma pequena amostra de entrada por uma camada neural simples; especificamente, recriamos o exemplo ilustrado na figura acima através do código seguinte, no qual implementamos uma camada neural com 5 entradas e 6 saídas que aplicaremos a dois exemplos de entrada.

In [8]:
torch.manual_seed(123)

# Cria 2 exemplos de treino com 5 features cada
batch_example = torch.randn(2, 5) 
batch_example

tensor([[-0.1115,  0.1204, -0.3696, -0.2404, -1.1969],
        [ 0.2093, -0.9724, -0.7550,  0.3239, -0.1085]])

In [9]:
layer = nn.Sequential(nn.Linear(5, 6), nn.ReLU())
out = layer(batch_example)
print(out)

tensor([[0.2260, 0.3470, 0.0000, 0.2216, 0.0000, 0.0000],
        [0.2133, 0.2394, 0.0000, 0.5198, 0.3297, 0.0000]],
       grad_fn=<ReluBackward0>)


In [10]:
# -1 torna invariante contra dimensões adicionais
mean = out.mean(dim=-1, keepdim=True) 
var = out.var(dim=-1, keepdim=True)

print("Mean:\n", mean)
print("Variance:\n", var)

Mean:
 tensor([[0.1324],
        [0.2170]], grad_fn=<MeanBackward1>)
Variance:
 tensor([[0.0231],
        [0.0398]], grad_fn=<VarBackward0>)


### Ilustrando o cálculo

![](https://camo.githubusercontent.com/1d353a009cf6ec3a2ea918b62bfedc10b565e0d17836366fc0e5dffca6156715/68747470733a2f2f73656261737469616e72617363686b612e636f6d2f696d616765732f4c4c4d732d66726f6d2d736372617463682d696d616765732f636830345f636f6d707265737365642f30362e77656270)

In [11]:
# Aplica a normalização de camada aos resultados da camada anterior
# A operação consiste em subtrair a média e dividir pelo desvio padrão.
out_norm = (out - mean) / torch.sqrt(var)
print("Normalized layer outputs:\n", out_norm)

mean = out_norm.mean(dim=-1, keepdim=True)
var = out_norm.var(dim=-1, keepdim=True)

# melhora a visualização
torch.set_printoptions(sci_mode=False)
print("Mean:\n", mean)
print("Variance:\n", var)

Normalized layer outputs:
 tensor([[ 0.6159,  1.4126, -0.8719,  0.5872, -0.8719, -0.8719],
        [-0.0189,  0.1121, -1.0876,  1.5173,  0.5647, -1.0876]],
       grad_fn=<DivBackward0>)
Mean:
 tensor([[    -0.0000],
        [     0.0000]], grad_fn=<MeanBackward1>)
Variance:
 tensor([[1.0000],
        [1.0000]], grad_fn=<VarBackward0>)


### Layer Norm (módulo)

In [12]:

class LayerNorm(nn.Module):
    def __init__(self, emb_dim):  # valores na dimensão do embedding
        super().__init__()
        self.eps = 1e-5
        # isso torna os valores treináveis
        self.scale = nn.Parameter(torch.ones(emb_dim))  
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)  # eps evita divisão por zero
        return self.scale * norm_x + self.shift

        # shift não tem efeito aqui pois adiciona apenas zero; será relevante mais tarde durante o treinamento,
        # permitindo que a rede desfaça essa normalização; similarmente para scale

### Layer Norm (aplicação)

In [13]:
ln = LayerNorm(emb_dim=5)
out_ln = ln(batch_example)
out_ln

tensor([[ 0.5528,  1.0693, -0.0223,  0.2656, -1.8654],
        [ 0.9087, -1.3767, -0.9564,  1.1304,  0.2940]], grad_fn=<AddBackward0>)

In [14]:
mean = out_ln.mean(dim=-1, keepdim=True)
var = out_ln.var(dim=-1, unbiased=False, keepdim=True)

print("Mean:\n", mean)
print("Variance:\n", var)

Mean:
 tensor([[    -0.0000],
        [     0.0000]], grad_fn=<MeanBackward1>)
Variance:
 tensor([[1.0000],
        [1.0000]], grad_fn=<VarBackward0>)


### Observações

- **Escala e deslocamento:** além da normalização (subtração da média e divisão pela variância), são introduzidos dois parâmetros treináveis, `scale` e `shift`. Inicialmente eles não alteram o valor (multiplicação por 1 e adição de 0), mas podem ser ajustados durante o treinamento para otimizar a performance do modelo.  
- **Variância enviesada:** ao calcular a variância é usado `unbiased=False`, ou seja, a fórmula $\frac{\sum_i(x_i-\bar{x})^2}{n}$. Essa escolha produz uma estimativa enviesada, mas em dimensões de embedding muito grandes a diferença entre $n$ e $n-1$ é insignificante. O GPT‑2 foi treinado com essa configuração, portanto o mesmo padrão é adotado para garantir compatibilidade com pesos pré‑treinados.  
O texto conclui convidando à experimentação prática do `LayerNorm`.

# Resumo

- **Arquitetura GPT**  
  - Bloco transformer auto‑regressivo (self‑attention + MLP).  
  - Pré‑treinado em previsão de próximo token; fine‑tune para tarefas específicas.  
  - Escalável: mais camadas, cabeças e dimensões → GPT‑2/3/4.
- **Normalização de Camada**  
  - Estabiliza ativações sem depender do batch (útil em sequências variáveis).  
  - Colocada após cada sub‑camada e antes da soma residual.  
  - Melhora convergência, evita saturação de ativação e mantém gradientes adequados.
- **Impacto**  
  - Modelos GPT com LayerNorm treinam mais rápido, são mais robustos e mantêm desempenho consistente em geração de texto.

# Próximos Passos

![](https://camo.githubusercontent.com/8ff24e3fedda0855855eacb08b6b1f78eac62471cef72b19539c068ec1c3bcda/68747470733a2f2f73656261737469616e72617363686b612e636f6d2f696d616765732f4c4c4d732d66726f6d2d736372617463682d696d616765732f636830345f636f6d707265737365642f30372e77656270)